In [31]:
!pip install comet_ml pytorch-lightning torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import os

import comet_ml
import torch.nn as nn
import pytorch_lightning as pl
import torch
import torch.nn.functional as F

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

In [69]:
class Model(pl.LightningModule):
  def __init__(self, number_of_filters, learning_rate):
    super().__init__()

    self.save_hyperparameters()

    self.learning_rate = learning_rate

    self.val_correct_counter = 0
    self.val_total_counter = 0
    self.val_batch_counter = 0

    self.hidden0 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=number_of_filters, kernel_size=4),
        nn.LeakyReLU(0.2)
    )

    self.hidden1 = nn.Sequential(
        nn.Conv2d(number_of_filters, number_of_filters * 4, kernel_size=4),
        nn.LeakyReLU(0.2)
    )

    self.hidden2 = nn.Sequential(
        nn.Conv2d(number_of_filters * 4, number_of_filters, kernel_size=4),
        nn.LeakyReLU(0.2)
    )

    self.hidden3 = nn.Sequential(
        nn.Linear(5776, 1000),
        nn.LeakyReLU(0.2)
    )

    self.hidden4 = nn.Sequential(
        nn.Linear(1000, 200),
        nn.LeakyReLU(0.2)
    )

    self.hidden5 = nn.Sequential(
        nn.Linear(200, 10)
    )

  def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)

        x = torch.flatten(x, start_dim=1, end_dim=-1)

        x = self.hidden3(x)
        x = self.hidden4(x)
        x = self.hidden5(x)

        return x
  
  def training_step(self, batch, batch_idx):
      x, y = batch

      predicted = self.forward(x)
      loss = F.cross_entropy(predicted, y)

      self.logger.log_metrics({"train_loss": loss}, step=batch_idx)
      return loss
  
  def validation_step(self, batch, batch_idx):
      x, y = batch

      predicted = self.forward(x)
      loss = F.cross_entropy(predicted, y)

      # to calculate the test accuracy
      self.val_correct_counter += int((torch.argmax(predicted, 1).flatten() == y).sum())
      self.val_total_counter += y.size(0)

      self.logger.log_metrics({"val_loss": loss}, step=batch_idx)
      return loss

  def on_validation_epoch_end(self):
      # outputs is an array with what you returned in validation_step for each batch
      # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}]

      avg_acc = 100 * self.val_correct_counter / self.val_total_counter

      self.val_correct_counter = 0
      self.val_total_counter = 0

      self.logger.log_metrics({"val_acc": avg_acc}, step=self.val_batch_counter)

      self.val_batch_counter += 1
  
  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [70]:
class invertColor():
    """
    Is used to inverse the color of a dataset when this dataset is in range[0;1]
    """
    def __call__(self, img):
        return self.invert(img)

    def invert(self, img):
        return 1 - img

In [71]:
class randomBlackToWhite():
    """
    Is used to transform some white pixels of the digit in black pixel
    """
    def __init__(self, prob):
      self.prob = prob

    def __call__(self, img):
        return self.randomBlackToWhite(img)

    def randomBlackToWhite(self, img, prob=0.9):
        return img * torch.distributions.bernoulli.Bernoulli(torch.tensor([self.prob])).sample(img.shape).squeeze(-1)

In [72]:
compose = transforms.Compose([
          transforms.RandomCrop(28, padding=8, pad_if_needed=True, fill=1, padding_mode='edge'),
          transforms.ToTensor(),
          randomBlackToWhite(0.7),
          invertColor(),
          transforms.Normalize((0.5,), (0.5,))
      ])

In [73]:
# Hyperparameters
hparam = {
    "learning_rate": 0.001,
    "number_of_filters": 16,
    "batch_size": 128 * 6,
    "optimizer": "Adam"
}

# Parameters
experiment_name = "First_Run"
checkpoint_folder = "./" + experiment_name + "_checkpoints/"
dataset_name = "MNIST_inv_color"
tags = ["cnn", "real", "inverse_color", "Bernoulli", "randomly_padded"]

In [74]:
# Init Model
model = Model(hparam["number_of_filters"], hparam["learning_rate"])

In [75]:
# Load Data
train_ds = MNIST(
    "data", train=True, download=True, transform=compose
)

train_loader = DataLoader(train_ds,  shuffle=True, batch_size=hparam["batch_size"])

eval_ds = MNIST(
    "data", train=False, download=True, transform=compose
)

eval_loader = DataLoader(train_ds, batch_size=hparam["batch_size"])

In [76]:
# init logger
comet_logger = CometLogger(
    api_key="hm3iPCrv0LDXFj8AHNO0uhBP4",
    rest_api_key="hm3iPCrv0LDXFj8AHNO0uhBP4",
    project_name="mnist-proseminar",
    experiment_name=experiment_name,
)

comet_logger.experiment.log_parameters(hparam)
comet_logger.experiment.add_tags(tags=tags)
comet_logger.experiment.log_dataset_info(name=dataset_name)

INFO:pytorch_lightning.loggers.comet:CometLogger will be initialized in online mode
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/luposx/mnist-proseminar/891d064210ed4a4695a5fa227170d906



In [77]:
# Initalize the trainer
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_folder)
trainer = pl.Trainer(# fast_dev_run=True,
                      callbacks=checkpoint_callback,
                      accelerator="gpu",
                      max_epochs=4,
                      logger=comet_logger)
trainer.fit(model, train_loader, eval_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | hidden0 | Sequential | 272   
1 | hidden1 | Sequential | 16.4 K
2 | hidden2 | Sequential | 16.4 K
3 | hidden3 | Sequential | 5.8 M 
4 | hidden4 | Sequential | 200 K 
5 | hidden5 | Sequential | 2.0 K 
---------------------------------------
6.0 M     Trainable params
0         Non-trainable params
6.0 M     Total params
24.049    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/luposx/mnist-proseminar/891d064210ed4a4695a5fa227170d906
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [28]        : (0.260108083486557, 2.1972951889038086)
COMET INFO:     train_loss [316] : (0.24508923292160034, 2.3009347915649414)
COMET INFO:     val_acc [5]      : (9.375, 90.52)
COMET INFO:     val_loss [318]   : (0.15661071240901947, 2.3031492233276367)
COMET INFO:   Others:
COMET INFO:     Name         : First_Run
COMET INFO:     dataset_info : MNIST_inv_color
COMET INFO:   Parameters:
COMET INFO:     batch_size      

In [ ]:
# log checkpoints
comet_logger.experiment.log_asset_folder(folder=checkpoint_folder)
comet_logger.experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/luposx/mnist-proseminar/f8e8db1f7f0d464d847a395a56bfdd7a

COMET WARNING: The default value for the log_file_name parameter will change from False to True in a future version. Explicitly pass log_file_name=True or log_file_name=False to disable this warning
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https